In [1]:
import re
import pandas as pd

In [2]:
with open("Lærkeskellet.txt","r",encoding='utf8') as f:
    body = f.read()
    print(body)


Salgspriser - alle boliger - Lærkeskellet, 2000

Adresse                                         Type           Aktuel
                                                               m²
Lærkeskellet 18, 2000 Frederiksberg             Ejerlejlighed
                                                4.575.000      140
Fri handel     15-05-2018                       1.026.500      138

Fri handel     27-07-1994                       Ejerlejlighed  140
                                                4.800.000      140
Lærkeskellet 6, 2000 Frederiksberg              3.070.000
                                                907.133        140
Fri handel     16-11-2017                                      140
                                                Ejerlejlighed  140
Fri handel     31-01-2012                       3.495.000      138
                                                               138
Fri handel     18-11-1994                       Ejerlejlighed
                             

In [3]:
addresses = re.findall('Lærkeskellet \d{1,2},', body)
square_meters = re.findall(' \d{2,3}\n', body)
dates = re.findall('\d\d-\d\d-\d\d\d\d', body)
prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)

I need to be able to find the addresses first and then find the dates for those addesses. Otherwise I can't line up the address, salesdate. 

The salesdate and price have the exact same number of rows, so those already lign up. But one or mole salesdates and prices can be part of the same address.

In [15]:
streetnames = [
    'Ahornskellet',
    'Askeskellet',
    'Birkeskellet',
    'Bøgeskellet',
    'Egeskellet',
    'Lindeskellet',
    'Lærkeskellet',
    'Pileskellet',
    'Plantanskellet',
    'Poppelskellet',
    'Tjørneskellet'
]
rows = []
for street in streetnames:
    with open(f"{street}.txt","r", encoding='utf8') as f:
        body = f.read()
    addresses = re.findall(f'{street} ' + '\d{1,3}, [0-9a-z., ]*2000', body)
    address_chuncks = re.split(f'{street} '+ '\d{1,2},', body)

    prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)
    square_meters = re.findall(' \d{2,3}\n', body)
    
    
    price_cnt = 0
    for address, m2, text in zip(addresses, square_meters, address_chuncks[1:]):
        m2 = int(m2.replace('\n',''))
        dates = re.findall('\d\d-\d\d-\d\d\d\d', text)
        for date in dates:
            try:
                price = prices[price_cnt]
                price = int(price.replace('.',''))
                
            except:
                break

            m2_price = round(price/m2)
            rows.append([address.replace(',',' ').replace('.','').replace('2000',''), date, price, m2, m2_price])
            price_cnt += 1

df = pd.DataFrame(columns=[
        'address',
        'salesDate',
        'price',
        'm2',
        'm2Price'
    ],
    data=rows)
len(df)

301

In [16]:
df

,address,salesDate,price,m2,m2Price
0,Ahornskellet 14 2,22-10-2019,2450000,62,39516
1,Ahornskellet 14 2,15-12-2011,1530000,62,24677
2,Ahornskellet 14 2,02-04-2008,1425000,62,22984
3,Ahornskellet 6 2,06-10-2019,3900000,111,35135
4,Ahornskellet 6 2,15-11-2005,3295000,111,29685
...,...,...,...,...,...
296,Tjørneskellet 4 2,09-01-1996,1272462,111,11464
297,Tjørneskellet 16 2,16-02-1998,1170000,113,10354
298,Tjørneskellet 16 2,24-05-1996,894062,113,7912
299,Tjørneskellet 8 st,08-05-1994,537851,122,4409


In [17]:
df.to_csv('../../data/real-estate/sales.csv')

In [24]:
pd.DataFrame(
    df.address.unique(),
    columns=['address']
).to_csv('../../data/real-estate/addresses.csv')

In [ ]:
pd.DataFrame(
    df.address.unique(),
    columns=['address']
).to_csv('../../data/real-estate/squaremeters.csv')